In [ ]:
import sys as _sys
import os

currentPath = os.path.abspath(os.getcwd())

split = currentPath.split("Cshells")
if len(split)<2:
    print("Please rename the repository 'Cshells'")
    raise ValueError
pathToPythonScripts = os.path.join(split[0], "Cshells/python/")
pathToModels = os.path.join(split[0], "Cshells/data/models")
pathToOutputs = os.path.join(split[0], "Cshells/output")

_sys.path.insert(0, pathToPythonScripts)

In [ ]:
import MeshFEM
import ElasticRods

import average_angle_linkages
from bending_validation import suppress_stdout as so
import elastic_rods
import math
import numpy as np
import pickle
import torch

from CShell import CShell
from CurvesDoFOptimizer import CurvesDoFOptimizer
from InteropGH import Interop
from open_average_angle_linkage import open_average_angle_linkage
import py_newton_optimizer
from vis.fields import ScalarField
from VisUtils import ConvergencePlotsVisualizer, PlotStackedConvergencePlots

torch.set_default_dtype(torch.float64)
    
def ToNumpy(tensor):
    return tensor.cpu().detach().clone().numpy()

PI = math.pi

# Initialization

In [ ]:
modelName = "tower_circ_bnd"

with open(os.path.join(pathToModels, modelName, "flat_initial.p"), 'rb') as f:
    flatLinkage = pickle.load(f)
    
with open(os.path.join(pathToModels, modelName, "deployed_initial.p"), 'rb') as f:
    depLinkage = pickle.load(f)
    
linkagesGuess = {
    "flat": flatLinkage,
    "deployed": depLinkage,
}

In [ ]:
with open(os.path.join(pathToModels, modelName, "cshell_initial.p"), 'rb') as f:
    dictCShell = pickle.load(f)
    
cshell = CShell(
    dictCShell["curvesDoF"], dictCShell["nJ"], dictCShell["curves"], dictCShell["curvesFamily"], dictCShell["nCPperRodEdge"], 
    dictCShell["alphaTar"], dictCShell["mult"], dictCShell["subdivision"], symmetry=dictCShell["symmetry"],
    attractionMesh=dictCShell["attractionMesh"], targetMesh=dictCShell["targetMesh"],
    rodMaterial=dictCShell["flatLinkage"].homogenousMaterial(), optimizeAlpha=True, useSAL=True, 
    linkagesGuess=linkagesGuess, flatOnly=False,
)

In [ ]:
cshell.flatView.show()

In [ ]:
cshell.deployedView.show()

# Optimize


In [ ]:
screenshot = None

pathToFolderJSON = os.path.join(pathToOutputs, "{}/optimization".format(modelName))
if not os.path.exists(pathToFolderJSON):
    os.makedirs(pathToFolderJSON)
pathToSaveJSON = os.path.join(pathToFolderJSON, modelName + "_{}_.json")

cshell.linkageOptimizer.scaleJointWeights(jointPosWeight=0.6, featureMultiplier=200.0) # The inner joints are free to slide
cshell.linkageOptimizer.setHoldClosestPointsFixed(False)
cshell.SetWeights(beta=1.0e6, gamma=0.0, smoothingWeight=1.0e-2, rlRegWeight=0.0, cpRegWeight=5.0e2)

cdo = CurvesDoFOptimizer(cshell, "knitro_lbfgs.opt", minAngle=None)
optDoF, optAlpha = cdo.OptimizeDoF(
    numSteps=5000, useCB=True, maxEqSteps=50, computeGradMags=False,
    ftol=1.0e-5, ftol_iters=3, visDeviations=True,
    saveGeometryPath=pathToSaveJSON, saveGeometryFrequency=3
)
nIterations = len(cshell.optimizationCallback.iterateData)

In [ ]:
print("There has been {} iterations".format(nIterations))

In [ ]:
cpv = ConvergencePlotsVisualizer(cshell.optimizationCallback)
cpv.PlotObjective(os.path.join(pathToFolderJSON, "optimVals.png"), "Convergence plot (objectives)", plotAll=True, wrtTime=True)
cpv.PlotGradMags(os.path.join(pathToFolderJSON, "optimGradMags.png"), "Convergence plot (gradient magnitudes)", wrtTime=False)

In [ ]:
tsf = cshell.linkageOptimizer.target_surface_fitter
l0 = np.linalg.norm(np.max(tsf.V, axis=0) - np.min(tsf.V, axis=0))
print("Initial target deviation")
for key in cpv.dpsMetric["TargetDeviationReport"][0].keys():
    print("{}: {:.2f}%".format(key, cpv.dpsMetric["TargetDeviationReport"][0][key] * 100 / l0))
print("\nFinal target deviation")
for key in cpv.dpsMetric["TargetDeviationReport"][-1].keys():
    print("{}: {:.2f}%".format(key, cpv.dpsMetric["TargetDeviationReport"][-1][key] * 100 / l0))

In [ ]:
pathToSaveFolder = os.path.join(pathToOutputs, "torus_symmetric/optimization")

with open(os.path.join(pathToSaveFolder, "objectivesCShell.p"), 'wb') as f:
    pickle.dump(cpv.dpsObjective, f)

with open(os.path.join(pathToSaveFolder, "metricsCShell.p"), 'wb') as f:
    pickle.dump(cpv.dpsMetric, f)

with open(os.path.join(pathToSaveFolder, "optimizationTimings.p"), 'wb') as f:
    pickle.dump(cpv.cumTimes, f)

with open(os.path.join(pathToSaveFolder, "flat_optimized.p"), 'wb') as f:
    pickle.dump(cshell.flatLinkage, f)

with open(os.path.join(pathToSaveFolder, "deployed_optimized.p"), 'wb') as f:
    pickle.dump(cshell.deployedLinkage, f)

with open(os.path.join(pathToSaveFolder, "cshell_optimized.p"), 'wb') as f:
    pickle.dump(cshell.GetCShellParams(), f)

# Produce convergence plots

In [ ]:
with open(os.path.join(pathToSaveFolder, "objectivesCShell.p"), 'rb') as f:
    dpsObjectives = pickle.load(f)

with open(os.path.join(pathToSaveFolder, "optimizationTimings.p"), 'rb') as f:
    dpsTimes = pickle.load(f)

listObjectivesNames = ['ElasticEnergyDeployed', 'TargetFitting', 'RestCurvatureSmoothing', 'LaplacianCP']
listObjectivesLabels = ['Elastic energy', 'Target fitting', 'Curvature smoothing', 'Laplacian']

listFN = [
    os.path.join(pathToSaveFolder, "cv_plot_nostack.png"),
    os.path.join(pathToSaveFolder, "cv_plot_no_text.png")
]
    
listCombos = [
    [True, False, False, True],
    [False, True, True, False]
]

for fn, combo in zip(listFN, listCombos):
    
    PlotStackedConvergencePlots(
        dpsObjectives, dpsTimes, listObjectivesNames, listObjectivesLabels, 
        showText=combo[0], againstTime=False, normalizeWithInit=False,
        filename=fn, transparent=True, logscale=True, showGrid=False, removeTicks=combo[1],
        preserveLegend=False, removeLegendBox=combo[2], removePlot=combo[3]
    )